# Run-level analyses
In this notebook, we will explain how to aggregate data from different fMRI runs using "run-level analyses". We will use both simulated and real data to explain this concept and show how to implement this in Python as well as FSL.

If you haven't done the other two notebooks (`linux_and_the_CMD.ipynb` and `first_level_analyses.ipynb` yet), please go through these first.

**What you'll learn**: after this lab, you'll be able to ...

* set up a run-level model in FSL FEAT

**Estimated time needed to complete**: 1-2 hours

## Run-level analyses
More often than not, researchers split their experiment across different fMRI *runs* (a period of continuous fMRI acquisition). These runs may be grouped within a particular *session* (a set of MRI scans within a particular period that the participant is in the scanner) or split across different sessions (e.g., run 1 and 2 are done on day 1, and run 3 and 4 are done on day 2).

<div class='alert alert-info'>
<b>ToDo</b> (1 point): Name one reason why researchers often rather split their experiment across different runs than having a single (longer) run. 
</div>

YOUR ANSWER HERE

As discussed before, splitting your experiment in different runs (across multiple sessions) generates a new "level" within data. If we analyzed each run/session in separate first-level models, we need to somehow aggregate (or average) the effects ($c\hat{\beta}$) and their variance ($\mathrm{var}[c\hat{\beta}]$) across these different runs. **If your experiment indeed contains multiple runs, then the next step in the "summary statistics" approach will be to combine their results**. 

Before discussing how the summary statistics approach would be implemented for fMRI data with multiple runs, let's focus on what a "traditional" hierarchical/multilevel model for this type of data would look like.

### The traditional multilevel model
In traditional hierarchical/multilevel (frequentists) GLM models, the data ($y$) and design matrices ($\mathbf{X}$) across different levels are simply concatenated. For example, for single-subject fMRI data with multiple runs, the signals ($y$) and design matrices ($\mathbf{X}$) are concatenated in time.

We'll show you how this is done for some example data. For example, suppose we have an experimental paradigm with two conditions ("A" and "B"), which we spread over 4 runs of 200 seconds/volumes (assuming a TR of 1 for simplicity).

We'll generate such a signal below:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from niedu.utils.nii import simulate_signal  # ignore the warning!

duration = 200
onsets = np.linspace(0, duration, 10, endpoint=False)
n_run = 4
Xs, ys = [], []
for run in range(n_run):  # simulate 4 signals / design matrices
    y, X = simulate_signal(
        onsets=onsets, conditions=np.tile(['A', 'B'], len(onsets) // 2),
        TR=1, duration=duration, params_canon=[2, 0.5], std_noise=2, plot=False, rnd_seed=run
    )
    ys.append(y)
    Xs.append(X[:, :3])  # remove tderivs

print("Number of signals: %i" % len(ys))
print("Size of signal for each run: %i\n" % ys[0].size)

print("Number of design matrices: %i" % len(Xs))
print("Shape of design matrix per run: %s" % (Xs[0].shape,))

Let's plot the signals from the separate runs:

In [ ]:
plt.figure(figsize=(7, 8))
for i in range(n_run):
    plt.plot(ys[i] + i*10, np.arange(ys[i].size))

plt.ylim(ys[i].size, 0)    
plt.xticks(np.arange(n_run) * 10, np.arange(1, n_run+1))
plt.ylabel("Time (sec.)", fontsize=20)
plt.xlabel("Run", fontsize=20)
plt.grid()
plt.show()

To create a "traditional" multilevel model, we need to concatenate the signals ($y$) in time:

In [ ]:
y = np.concatenate(ys)
print("Size of signal: %i" % y.size)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Let's plot the concatenated signal (but color-code the runs)
plt.figure(figsize=(3, 16))
for i, ytmp in enumerate(ys):
    t = np.arange(duration * i, duration * (i + 1))
    plt.plot(ytmp, t, c=colors[i])

plt.ylim(y.size, 0)
plt.ylabel("Time (sec.)", fontsize=20)
plt.xlabel("Signal amplitude (A.U.)", fontsize=20)
plt.grid()
plt.show()

Similarly, we need to stack the run-wise design matrices, but these are stacked both vertically (in time) and horizontally (as separate predictors). In other words, each predictor (including the intercept!) in each run gets its own column (i.e., predictor) in our multilevel design matrix:

In [ ]:
X = np.zeros((len(Xs) * duration, len(Xs) * 3))
for i in range(len(Xs)):
    t = np.arange(duration * i, duration * (i + 1))
    X[t, i*3:(i+1)*3] = Xs[i]
    
# number of columns = number of conditions * number of runs
print("Shape of concatenated design matrix: %s" % (X.shape,))

It's probably easier to understand it if it's plotted. Below, we'll plot the concatenated signal ($y_{all}$) and the concatenated design matrix ($X_{all}$) in a single figure:

In [ ]:
fig, axes = plt.subplots(ncols=len(Xs) + 1, figsize=(15, 15), sharey=True, sharex=False)

for i, ytmp in enumerate(ys):
    t = np.arange(duration * i, duration * (i + 1))
    axes[0].plot(ytmp, t, c=colors[i])
    axes[0].set_ylim(duration * n_run, 0)
    axes[0].spines['right'].set_visible(False)
    axes[0].spines['top'].set_visible(False)
    
    axes[i].grid()

    axes[i+1].set_title(r"$X_{run\ %i}$" % (i+1), fontsize=25)
    for ii in range(3):
        pred = X[:, (i*3)+ii]
        t = np.arange(duration * n_run)
        axes[i+1].plot(pred + (ii*2), t, c=colors[i], lw=2)
    
    axes[i+1].spines['right'].set_visible(False)
    axes[i+1].spines['top'].set_visible(False)
    axes[i+1].set_xticks([0, 2, 4])
    axes[i+1].set_xticklabels(
        [r'$icept_{%i}$' % (i+1), r'$A_{%i}$' % (i+1),r'$B_{%i}$' % (i+1)], fontsize=15
    )

plt.figtext(0.61, 0.92, r'$X_{all}$', fontsize=30)
axes[0].set_title(r'$y_{all}$', fontsize=30, y=1.045)

axes[0].set_ylabel("Time (volumes)", fontsize=20)
axes[i+1].grid()
fig.show()

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points)
    
Given the concatenated signal (the variable <tt>y</tt>) and concatenated design matrices (the variable <tt>X</tt>), you can run a single GLM to get the parameters for the different conditions across the four runs (i.e., $icept_{1}$, $icept_{2}$, $A_{1}$, $A_{2}$, $B_{1}$, $B_{2}$, etc.). Do this for our data (using <tt>y</tt> and <tt>X</tt>) and store the estimated parameters (there should be 12) in a new variable named <tt>av_betas</tt>. 

Then, you can in fact specify a specific contrast vector that gives you the *average* effect (i.e., $\hat{\beta}_{A}$ or $\hat{\beta}_{B}$). Try to think of which particular contrast (which should also be of size 12) would "compute" the average effect. Create a separate contrast vector for the average effect of A (store this in a variable named <tt>cvec_a</tt>) and the average effect of B (store this in a variable named <tt>cvec_b</tt>).

Hint: you can of course check whether your contrast-vectors in fact yield the mean effect ($\hat{\beta}$) by comparing it to the result when you manually compute the mean!
</div>

In [ ]:
from numpy.linalg import inv

# Implement the ToDo here!
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
from niedu.tests.nii.week_5 import test_ffx_glm
test_ffx_glm(X, y, av_betas, cvec_a, cvec_b, n_run)

Now, the previous ToDo only concerned the estimation of the run-average *effects* (and the contrast), but not the estimation of the *variance of the run-average effect*. The exact way this is computed actually depends on whether you want to use a fixed-effect, random-effect, or mixed-effects approach. We will discuss this later in this notebook. First, let's take a look at how fMRI studies usually deal with multilevel data: using the summary statistics approach. 

### The summary statistics approach
Most fMRI studies dealing with multilevel data often do not use the traditional multilevel model, but use a slightly different approach: instead of running one single GLM for the concatenated data, it runs a GLM *per* unit within each level and subsequently "averages" the contrast estimates using the "summary statistics" approach. For example, for multi-run data, a first-level model for each separate run is evaluated (instead of a single, concatenated model). This is also the approach FSL takes.

After estimating a first-level model for each run separately, the summary statistics approach uses the results (i.e., $c\hat{\beta}$, representing the "summary statistics") of the previous level as the "data-to-be-explained" (i.e., $y$) within the current level. Using a "run-level" GLM with a new design matrix, the data from the previous level can be aggregated as desired, where the results (i.e., $c\hat{\beta}$) of the *current* level GLM represent the aggregated data. This is both the case for aggregating results across runs (in run-level analyses) and across subjects (in group-level analyses). The global idea of this summary statistics approach is visualized in the figure below.

![](https://docs.google.com/drawings/d/e/2PACX-1vQxCH3WU3nTqFlHUZb49rf9zioivGQ-flVfRpwmXQx7OF5Wm_1T6gFMYQqpqt-NPITNHUaRoVYEREgT/pub?w=1442&h=1168)

To put it another way, to aggregate first-level results across runs, we are going to construct another GLM at the "run-level". To distinguish components of run-level GLMs from first-level GLMs, we will add a superscript asterisk ($^{*}$) to the run-level GLM components.

So, in our run-level GLM, the first-level results ($c\hat{\beta}$) become our new dependent variable ($y^{*}$): 

\begin{align}
\mathbf{y}^{*} = c\hat{\beta}
\end{align}

which is, similar to first-level GLMs, modeled using a (run-level) GLM with a specific run-level design matrix, $\mathbf{X}^{*}$, and associated run-level parameters, $\hat{\beta}^{*}$:

\begin{align}
\mathbf{y}^{*} = \mathbf{X}^{*}\beta^{*} + \epsilon^{*}
\end{align}

These run-level parameters are usually estimated using OLS:

\begin{align}
\hat{\beta}^{*} = (\mathbf{X}^{*T}\mathbf{X}^{*})^{-1}\mathbf{X}^{*T}\mathbf{y}^{*}
\end{align}

Importantly, the way you specify the run-level design matrix, $\mathbf{X}^{*}$, dictates the way the data is aggregated, which we'll come back to later. For now, it is important to understand that the summary statistics approach entails using the results from the previous level ($c\hat{\beta}$) as the target/dependent variable ($\mathbf{y}^{*}$) in the current level.

In what follows, we'll explain the process using the previously simulated dataset (with two conditions, "A" and "B"). First of all, we need to run our "first-level" models for the four runs separately.

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): Within a loop, compute the contrast against baseline for both condition "A" ($\beta_{A} > 0$) and condition "B" ($\beta_{B} > 0$) per run and store these results in the pre-allocated <tt>runwise_cb</tt> array, where the first row should represent the $\beta_{A} > 0$ results and the second row should represent the $\beta_{B} > 0$ results. 

Use the variables <tt>Xs</tt> and <tt>ys</tt>, which are both <em>lists</em> of length 4, corresponding to the four runs. Each element in <tt>Xs</tt> contains a $200$ (N) $\times\ 2$ (P) array. Each element in <tt>ys</tt> contains a 1D array of size $200$ (N).
</div>

In [ ]:
''' Implement your ToDo here. '''
# array has shape: num contrasts x N_runs
runwise_cb = np.zeros((2, len(Xs)))

# Implement your loop here, in which you fill the 
# runwise_cb array with the different 
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_5 import test_ss_ffx_glm
test_ss_ffx_glm(Xs, ys, runwise_cb)

In case you couldn't figure out the previous ToDo, we'll load in the (approximately) correct `runwise_cb` values below, so we can continue the explanation (note that this will overwrite your own answer, and will give an error if you run the test cell above, but during grading we will only use your own implementation).

In [ ]:
runwise_cb = np.load('runwise_cb_answer.npy')

Okay, so within our summary statistics approach, the $c\hat{\beta}$ values will become our new data ($y^{*}$). For now, let's only focus on the condition "A" against baseline contrast (the first row in `runwise_cb`). Let's redefine this as `y_rl` ("y, run-level"):

In [ ]:
y_rl = runwise_cb[0, :]

We have defined our dependent variable, but what should our run-level design matrix ($\mathbf{X}^{*}$) be? This, of course, depends on what you are interested in. Often, people simply want to aggregate the results across different runs into a single "average" estimate. 

It turns out that using a vector of ones as your design matrix ($\mathbf{X}^{*}$) will yield run-level parameter estimates ($\hat{\beta}^{*}$) that correspond to the average of your first-level results. So, when you use a vector of ones as your run-level design matrix, $\mathbf{X}^{*}$ ...

\begin{align}
\mathbf{X}^{*} = \begin{bmatrix}
             1 \\
             1 \\
             \vdots \\
             1
         \end{bmatrix}
\end{align}

then the parameter of this run-level GLM will correspond to the average of the first-level results ($c\hat{\beta}$):

\begin{align}
\mathrm{average}(c\hat{\beta}) = \hat{\beta}^{*} = (\mathbf{X}^{*T}\mathbf{X}^{*})^{-1}\mathbf{X}^{*T}y^{*}
\end{align}

So, why is this design &mdash; in which the average is just a vector of ones for each run &mdash; modelling the average of the first-level results? Well, let's investigate this using our previously simulated data. Now, we know that our GLM (with a vector of ones) should give us the mean. Let's calculate the mean 'manually' so that we can check later whether the GLM gives us the same answer.

In [ ]:
manual_mean = np.mean(y_rl)
print(manual_mean)

<div class='alert alert-warning'>
<b>ToDo</b> (1 point): Create a design-matrix of shape $(4, 1)$, in which the single column contains all ones (you can use the <tt>np.ones</tt> function for this!). Then run linear regression using the variable <tt>y_rl</tt> as our target and the using the design-matrix you just created (representing the "$\mathbf{X}$"). Store the resulting parameter-value in a variable named <tt>glm_mean</tt>.  
</div>

In [ ]:
''' Implement your ToDo here.'''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo.'''
if isinstance(glm_mean, np.ndarray):
    ans = glm_mean[0]
else:
    ans = glm_mean

np.testing.assert_almost_equal(ans, manual_mean)
print("Well done!")

If you've done the previous ToDo correctly, you've seen that, as expected, the parameter calculated by the GLM when using a predictor with all ones (a run-level intercept, basically) reflects the mean of the dependent-variable. In fact, in the context of the GLM &mdash; which aims to minimize the residuals between the predictor(s) and the target &mdash; this makes sense! We show you this in the plot below:

In [ ]:
rnge = np.max(y_rl) - np.min(y_rl)
fig, axes = plt.subplots(ncols=2, figsize=(12, 11), sharey=True, sharex=True)
for i, ax in enumerate(axes):
    ax.plot(y_rl, np.arange(1, y_rl.size+1), marker='o', ms=12, lw=2)
    ax.plot(np.ones(n_run), np.arange(1, n_run+1), ls='--', lw=2)
    ax.legend([r'$Y^{*}$', r'$X^{*}$'], fontsize=15, frameon=False)
    ax.set_yticks(np.arange(1, n_run+1)[::-1])
    ax.set_yticklabels(np.arange(1, n_run + 1), fontsize=20)
    ax.grid()
    ax.set_xlim(0, np.max(y_rl) + 0.3 * rnge)
    ax.set_xlabel(r"Contrast-values ($c\beta$)", fontsize=20)

    if i == 0:
        ax.set_ylabel(r"Runs", fontsize=20)
        ax.set_title("Run-level GLM model", fontsize=25)
    else:
        ax.set_title("Run-level *fitted* GLM model", fontsize=25)
        ax.plot(np.repeat(manual_mean, n_run), np.arange(1, n_run+1), c='tab:orange', lw=3)
        for i in range(n_run):
            ax.plot((y_rl[i], manual_mean), (i+1, i+1), c='tab:red', ls='--', lw=3)
            
        ax.annotate(text='', xy=(0, 2.5), xytext=(manual_mean, 2.5), arrowprops=dict(arrowstyle='<->', lw=4))
        ax.text(0.5, 2.3, r'$\hat{\beta}^{*}$', fontsize=30)
        ax.legend([r'$Y^{*}$', r'$X^{*}$', r'$\hat{Y}^{*} = \hat{\beta}^{*}$', r'$residuals^{*}$'],
                  fontsize=15, frameon=False, loc='lower left')
        
fig.tight_layout()   

From the plot above, you can nicely see that the vector of ones nicely models the mean of the dependent variable (the first-level $\beta_{A} > 0$ contrasts). Just like in first-level models, we can specify particular **run-level** contrasts. Implicitly, we used a **run-level** contrast against baseline here (because for our single run-level predictor, when $c^{*} = [1]$, then $c\hat{\beta}^{*} = \hat{\beta}^{*}$), but other contrasts are definitely possible (we'll take a look at that later).

But what if we want to test more than one *first-level* contrast, eventually, at the group-level? For example, suppose we also want to test $\beta_{B} > 0$. Should we then just run a *separate* run-level analysis for each first-level contrast? You surely can, but often it's easier to do it in a single model in which we concatenate the contrast-values from the different contrasts. We do this below:

In [ ]:
# order: A1, A2, A3, A4, B1, B2, B3, B4
y_AB = np.concatenate([runwise_cb[0, :], runwise_cb[1, :]])

plt.figure(figsize=(8, 15))
plt.plot(runwise_cb[0, :], np.arange(n_run), marker='o', ms=12, lw=2)
plt.plot(runwise_cb[1, :], np.arange(n_run, n_run * 2), c='tab:green', marker='o', ms=12, lw=2)
plt.yticks(np.arange(n_run * 2), ['Run ' + str(i) for i in np.tile(range(1, n_run+1), 2)], fontsize=15)
plt.xlim(-1, 5)
plt.ylabel("Runs", fontsize=20)
plt.xlabel(r"Contrast-values ($c\hat{\beta}$)", fontsize=20)
plt.title("Concatenated contrast-values, A and B", fontsize=25)
plt.axvline(0, c='black', ls='--', lw=0.5)
plt.legend([r'$Y_{A}$', r'$Y_{B}$'], fontsize=20)
plt.gca().invert_yaxis()
plt.grid()
plt.show()

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): Now, given the concatenated data (the variable <tt>y_AB</tt>), can you come up with a design-matrix ($X$) that models both the mean of the contrast-values of A-against-baseline (first four values) and the contrast-values of B-against-baseline (last four values)? Name your design-matrix <tt>X_concat_data</tt> (hint: it needs two columns), which should be a 2D numpy array. Then, run linear regression on the concatenated data using your design-matrix (<tt>X_concat_data</tt>); you can check your answer by verifying that the two parameters from the run-level regression model are the same as the mean across the columns of the <tt>runwise_cb</tt> array.
</div>

In [ ]:
''' Implement your ToDo here. '''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_5 import test_concat_design
test_concat_design(n_run, X_concat_data)

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point): Right now, we only have the <em>first-level</em> contrasts against baseline to our disposal ($\beta_{A} > 0$ and $\beta_{B} > 0$). But suppose that I'm actually interested to see which voxels significantly activate in response to both stimuli across runs, i.e., $(\beta_{A} > 0)\ \&\ (\beta_{B} > 0)$. We can actually specify a particular <em>run-level</em> contrast that tests this. Create an array (with size 2) that implements this contrast vector, and store it in a variable named <em>cvec_A_and_B</em>.
</div>

In [ ]:
''' Implement your ToDo here. '''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the ToDo above. '''
from niedu.tests.nii.week_5 import test_cvec_A_and_B    
test_cvec_A_and_B(cvec_A_and_B)

<div class='alert alert-warning'>
    <b>ToDo</b> (optional, bonus point): Suppose that, for some reason, I'm interested in investigating whether there are voxels that respond more strongly to condition "A" in the first two sessions than the last two sessions. Given our first-level results (stored in <tt>y_AB</tt>), how should my run-level design matrix ($\mathbf{X}_{rl}$) and run-level contrast vector look like? For a bonus point, create this matrix and contrast vector below, run linear regression on the run-level data (<tt>y_AB</tt>) using your design matrix, and compute the run-level contrast using your contrast vector. Store the result of this run-level contrast (i.e., a single nubmer) in a variable named <tt>runlevel_cb_optional</tt>.
</div>

In [ ]:
''' Implement the (optional) ToDo here. '''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the ToDo above. '''
from niedu.tests.nii.week_5 import test_runlevel_cb_optional
test_runlevel_cb_optional(runlevel_cb_optional, y_AB)

### Fixed vs. random vs. mixed effects
Remember that we told you that hierarchical data, such as multi-subject fMRI data, is often analyzed with mixed models which compute the variance (uncertainty) of the effects using the variance estimates at each level within your model. This specific type of analysis, in which the final variance estimate is computed using variance components across all levels, is often called a **mixed-effects analysis**.

In the run-level models so far, though, we only focused on the run-level *effects* ($c\hat{\beta}_{rl}$), not their variance! In fact, for run-level analyses, it is common to "ignore" the run-level variance and simply average the first-level variance estimates across runs. In other words, this is treating the effect as "fixed" across runs and assuming the variance per run is similar (so that these variance estimates can be average). This specific analysis, in which the effect is assumed to be fixed across observations within an analysis level (e.g., across runs), is often called a **fixed-effects analysis**. Just like the mixed-effects analysis, they are all (variations on) the GLM with specific ways in which they estimate the variance component of our effects.

In the context of analysis of hierarchical fMRI data, **fixed-effects analyses** at the run-level are completely reasonable (at least, that's what most people do in practice). However, this is not true for the next level in our summary statistics approach: the group-level. Here, you actually need to incorporate the variance component from the group-level in order to draw valid population inferences. When you incorporate this group-level variance component, but ignore the lower-level variance components (i.e., at the first-level or run-level), this type of analysis is not a full mixed-effects analysis, but is often called a **random-effects analysis**. This is the approach that SPM takes (which is fine, given some assumptions). FSL, on the other hand, offers both random-effects as well as a full mixed-effects option, in which the latter is estimated by incorporating both first-level (or run-level) and group-level variance estimates (which approximates the "proper" mixed model without the summary statistics procedure). 

But let's not get distracted (group-level models is the topic of next week!) and continue with this notebook's last section on how to implement run-level (fixed-effects) analyses in FSL.

### Run-level analyses in FSL
In FSL, any analysis that is not at the first-level is called a "higher-level analysis". In this section, we are going to set up such a higher-level analysis with the goal to average our first-level `flocBLOCKED` results. In particular, we are going to do this specifically for the following contrasts:

* $4\times\beta_{\mathbf{face}} - \beta_{body} - \beta_{place} - \beta_{character} - \beta_{object} > 0$ contrast (this was contrast number 4 in our original first-level analysis)
* $4\times\beta_{\mathbf{place}} - \beta_{body} - \beta_{face} - \beta_{character} - \beta_{object} > 0$ contrast (this was contrast number 5 in our original first-level analysis).

<div class='alert alert-warning'>
<b>ToDo</b> (not graded): Open FEAT and change the type of analysis from "First-level analysis" to "Higher-level analysis". 
</div>

You should see that the "Pre-stats" and "Registration" tabs become unavailable, as FEAT assumes these things have been done in the first-level analyses. 

<div class='alert alert-warning'>
<b>ToDo</b> (not graded): First, go to the "Stats" tab and change the analysis-type from "Mixed effects: FLAME 1" to "Fixed effects". 
</div>

Now, let's tell FEAT which data we want to analyze. For the upcoming ToDos, we're going to average the "face > other" and "place > other" contrasts. The two first-level analyses (one for each run) have been run already and are located here:

In [ ]:
import os
data_dir = os.path.join(os.path.expanduser("~"), 'NI-edu-data')
print("Starting download of FEAT directory (+- 287MB) ...\n")
!aws s3 sync --no-sign-request s3://openneuro.org/ds003965 {data_dir} --exclude "*" --include "derivatives/fsl/sub-03/flocBLOCKED/ses-2.feat/*"
print("\nDone!")

In [ ]:
feat_r1 = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'ses-1.feat')
print("The run 1 results are here: %s" % feat_r1)
feat_r2 = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'ses-2.feat')
print("The run 2 results are here: %s" % feat_r2)

<div class='alert alert-warning'>
<b>ToDo</b> (not graded): 

Change the setting "Inputs are lower-level FEAT directories" to "Inputs are 3D cope images from FEAT directories". Set the "Number of inputs" to 4. Then, press the "Select cope images" button and add the different first-level contrast estimate files. The order should be: "face > other" (session 1), "face > other" (session 2), "place > other" (session 1), and "place > other" (session 2):

\begin{align}
y^{*} = 
\begin{bmatrix}
c\hat{\beta}_{\mathrm{face>other,\ session\ 1}} \\
c\hat{\beta}_{\mathrm{face>other,\ session\ 2}} \\
c\hat{\beta}_{\mathrm{place>other,\ session\ 1}} \\
c\hat{\beta}_{\mathrm{place>other,\ session\ 2}} \\
\end{bmatrix}
\end{align}

Check the ToDo in section 4.1.4. to see which contrast ("COPE") numbers these first-level contrasts refer to.
Set the "Output directory" to the "week 5" folder. And, under the "Post-stats" tab, set the thresholding to "Uncorrected" at a threshold at $p < 0.005$.
</div>

Note that, instead of assuming that "Inputs are 3D cope images from FEAT directions", you could use the other option "Inputs are lower-level FEAT directories". This option allows you to apply a particular higher-level design ($\mathbf{X}^{*}$) and higher-level contrasts ($\mathbf{c}^{*}$) for *all first-level contrasts* at the same time. Here, we don't use this option as it is less clear what is happening "under the hood".  

<div class='alert alert-danger'>
<b>Assignment</b> (2 points)<br>

In the "Stats" tab, click on the "Full model setup" button. Now, you can directly specify your design matrix ($\mathbf{X}^{*}$) here in the "EVs" tab. Here, create a design matrix that allow you to average the first-level "face > other" and "place > other" contrasts within a (single) run-level GLM (hint: you might need to change the "Number of main EVs"). Then, go to the "Contrasts and F-tests" tab and create the contrasts that you need in order to average the "face > other" first-level contrast across runs and the "place > other" first-level contrast across runs. Give the EVs and contrasts sensible names.

When you're happy with your setup, save your setup in your <tt>week_5</tt> directory; give it the name <tt>setup_feat_runlevel</tt>. Then, add the following line to the text-cell below:

`![](setup_feat_runlevel.png)`

After adding this line and running the cell, the run-level design should be visible. If it is not, you probably saved the design in the wrong directory.
    
</div>

YOUR ANSWER HERE

## Checking out results from run-level FEAT analyses
We actually already ran the run-level analysis, of which the results are| downloaded in the cell below:

In [ ]:
print("Starting download of runlevel FEAT directory (+- 40MB) ...\n")
!aws s3 sync --no-sign-request s3://openneuro.org/ds003965 {data_dir} --exclude "*" --include "derivatives/fsl/sub-03/flocBLOCKED/runlevel_Week5.gfeat/*"
print("\nDone!")

runlevel_dir = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'runlevel_Week5.gfeat')
print(runlevel_dir)

<div class='alert alert-warning'>
    <b>ToDo</b>: Open a terminal and navigate to the above <tt>runlevel_Week5.gfeat</tt> directory (using <tt>cd</tt>). Then, print its contents to the terminal window (using <tt>ls</tt>)!
</div>

The `gfeat` suffix stands for "group FEAT" and is appended to each higher-level analysis output directory (i.e., any analysis that is not a first-level analysis).

In this `gfeat` directory, you should see similar files as you've seen in first-level FEAT directories: HTML-files with analysis summaries, files with design information, etc. 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): In your terminal, open the <tt>report.html</tt> file in the <tt>gfeat</tt> directory with Firefox, which should open a new Firefox window. Then, click on the "Registration summary" tab.
</div>

The registration tab conveniently shows us the "functional &rarr; standard space" registration results again, but now for all four runs at the same time. Here you can easily check for possible large registration differences between runs/sessions. Note that this particular transformation is only executed in higher-level analyses. While the (linear and non-linear) registration parameters are already *computed* in the first-level analysis, they are *applied* in the higher-level analysis.

<div class='alert alert-info'>
    <b>ToThink</b> (ungraded): Can you think of a reason why spatial resampling to standard space is postponed to the higher-level stage?
</div>


If you scroll further down, you see a figure with the caption "Sum of all input masks after transformation to standard space". This shows you how well the functional brain masks (delineating brain from non-brain matter, i.e., the skullstripping result for the different functional files) align. It uses a red-yellow colormap from 1 (red) to 4 (yellow), showing where all masks overlap (yellow) and where this is not the case (relatively red voxels). There are some voxels not included in all masks in orbitofrontal cortex (visible in the first row of brain slices), which might be caused by different amounts of signal dropout across runs, but that isn't much of a problem.

The figure below ("Unique missing-mask voxels") is a similar quality check, where it plots voxels missing in precisely *one* mask, which allows you to easily spot whether a single registration did not work as expected. Here, everything looks alright.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Click on the "Results" tab. Note that the image of the design matrix is missing, because we have deleted this (as it would give away the answer to the previous assignment). Now, click on the "Higher-level FEAT results" link, which will open the "post-stats" tab of the higher-level FEAT analysis. Here, you can see two figures, showing the (thresholded) results of our run-level contrasts ($c\hat{\beta}^{*}$).
</div>

Within the `gfeat1` directory, of particular interest is the `cope1.feat` directory. This subdirectory contains the actual *results* of the run-level analysis. The reason this directory is called `cope1.feat` is because the other input option ("Inputs are lower-level FEAT directories") allows you to apply the higher-level design to multiple lower-level contrasts at the same time, which would create separate `cope*.feat` subdirectories (`cope1.feat`, `cope2.feat`, `cope3.feat`, etc.). The "Inputs are 3D cope images from FEAT directories" option only creates a single directory which is by default called `cope1.feat`.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): In your terminal, navigate into the <tt>cope1.feat</tt> directory and print its contents to the terminal window.
</div>

Here, you see that this `cope1.feat` subdirectory has the exact same structure and files as the first-level `.feat` directories that we inspected earlier! 

Let's view the results in FSLeyes.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Close any active overlays (Overlay &rarr; Remove all). Then, click on File &rarr; Add standard &rarr; click on <tt>MNI152_T1_2mm_brain.nii.gz</tt>, which is the standard space image that FSL used as the group template. Then, add the <tt>thresh_zstat1.nii.gz</tt> image from the <tt>cope1.feat</tt> directory (i.e., <tt>runlevel.gfeat/cope1.feat/thresh_zstat1.nii.gz</tt>). This file corresponds to the (thresholded) fixed-effects "average" effect of our lower-level $\beta_{face} > 0$ contrasts.

Change the colormap to "Red-Yellow".
</div>

Viewing results from run-level analyses is not fundamentally different from first-level analyses, with one exception: the data is now in "standard space" (i.e., MNI152, 2mm space). This allows us to use *atlases* to connect the location of our effects to particular anatomical brain regions.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): In the top menu, click on Settings &rarr; Ortho View 1 &rarr; Atlas panel. This should open a new panel in between the Overlay list and the Location panel.
</div>

The Atlases panel by default has to to atlases loaded: the [Harvard-Oxford Cortical Structural atlas](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Atlases) and the Harvard-Oxford Subcortical Structural atlas, which are based on the individual segmentations of 37 T1-weighted scans. 

Importantly, these atlases are *probabilistic*, meaning that, for each voxel, they give a *probability* (or, actually, percentage) of belonging to a particular (set of) region(s). The exact percentage is based on the number of people (out of 37) in which that voxel belonged to that region. So, if, for a particular voxel, the atlas shows you "82% left amygdala", that means that that voxel was part of the left amygdala for 81% of those 37 people.

Consequently, most voxels actually have multiple labels (e.g., 81% left amygdala, 19% left hippocampus).

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded): Go to voxel $[25, 39, 25]$ and check out the regions listed in the atlas panel. Do you expect to find this region for this particular contrast/analysis?
</div>

<div class='alert alert-success'>
    <b>Tip!</b>
    Before handing in your notebooks, we recommend restarting your kernel (<em>Kernel</em> &rarr; <em>Restart & Clear Ouput</em>) and running all your cells again (manually, or by <em>Cell</em> &rarr; <em>Run all</em>). By running all your cells one by one (from "top" to "bottom" of the notebook), you may spot potential errors that are caused by accidentally overwriting your variables or running your cells out of order (e.g., defining the variable 'x' in cell 28 which you then use in cell 15).
</div>